In [1]:
import pandas as pd
import requests
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
import csv

In [2]:
base_url = "http://api.coingecko.com/api/v3"

<h1>Get crypto list</h1>

In [3]:
request_url = "/coins/markets"

params_dict = {"vs_currency" : "USD",
              "per_page" : 250,
              "page" : 1}

In [4]:
response = requests.get(base_url + request_url, params= params_dict)
if response.status_code != 200:
    print("Error")
data = response.json()

In [5]:
list_cryptos = []
for _dict in data:
    list_cryptos.append(_dict['id'])

In [6]:
list_cryptos

['bitcoin',
 'ethereum',
 'binancecoin',
 'ripple',
 'tether',
 'cardano',
 'dogecoin',
 'polkadot',
 'uniswap',
 'litecoin',
 'bitcoin-cash',
 'chainlink',
 'usd-coin',
 'vechain',
 'solana',
 'stellar',
 'theta-token',
 'filecoin',
 'tron',
 'wrapped-bitcoin',
 'monero',
 'binance-usd',
 'terra-luna',
 'neo',
 'klay-token',
 'iota',
 'aave',
 'pancakeswap-token',
 'eos',
 'cosmos',
 'bitcoin-cash-sv',
 'crypto-com-chain',
 'bittorrent-2',
 'ftx-token',
 'okb',
 'compound-usd-coin',
 'ethereum-classic',
 'matic-network',
 'compound-ether',
 'tezos',
 'maker',
 'algorand',
 'dai',
 'avalanche-2',
 'kusama',
 'thorchain',
 'cdai',
 'huobi-token',
 'compound-governance-token',
 'elrond-erd-2',
 'nem',
 'dash',
 'chiliz',
 'holotoken',
 'decred',
 'zcash',
 'havven',
 'celsius-degree-token',
 'hedera-hashgraph',
 'zilliqa',
 'enjincoin',
 'blockstack',
 'leo-token',
 'sushi',
 'amp-token',
 'nexo',
 'terrausd',
 'waves',
 'the-graph',
 'siacoin',
 'fei-protocol',
 'decentraland',
 'digiby

<h1>Get timeseries marketcap and volume data for a list of tokens, for the last year</h1>

In [40]:
with open('targets_lists/coin_list.csv', newline=None) as f:
    reader = csv.reader(f)
    coin_list=list(reader)

In [41]:
start_date = datetime.datetime(2020,4,21)
end_date = datetime.datetime(2021,4,21)
params_dict = {"vs_currency" : "usd",
              "from" : datetime.datetime.timestamp(start_date),
              "to" : datetime.datetime.timestamp(end_date)}

In [42]:
request_url = f"/coins/{coin_list[0][0]}/market_chart/range"
response = requests.get(base_url + request_url, params = params_dict)
if response.status_code != 200:
    print(f"API error on {coin_list[0][0]}")
else:
    coin_data = response.json()
    coin_marketcap_data_df = pd.DataFrame(data = coin_data["market_caps"], index=None, columns=["Date", coin_list[0][0]])
    coin_volume_data_df = pd.DataFrame(data = coin_data["total_volumes"], index=None, columns=["Date", coin_list[0][0]])
    print(f"Successfully saved {coin_list[0][0]} data")

if len(coin_list[0]) > 0:
    for coin_id in coin_list[0][1:]:
        request_url = f"/coins/{coin_id}/market_chart/range"
        response = requests.get(base_url + request_url, params = params_dict)
        if response.status_code != 200:
            print(f"API error on {coin_id}")
        else:
            coin_data = response.json()
            coin_mc = [day[1] for day in coin_data["market_caps"]]
            coin_v = [day[1] for day in coin_data["total_volumes"]]
            coin_marketcap_data_df[[f'{coin_id}']] = pd.DataFrame(coin_mc, index=None)
            coin_volume_data_df[[f'{coin_id}']] = pd.DataFrame(coin_v, index=None)
            print(f"Successfully saved {coin_id} data")
        
coin_marketcap_data_df.to_csv(f"coin_data/coin_marketcap_data.csv", index = False)
coin_volume_data_df.to_csv(f"coin_data/coin_volume_data.csv", index = False)

Successfully saved bitcoin data
Successfully saved ethereum data
Successfully saved binancecoin data
Successfully saved huobi-token data


<h1>Get timeseries volume data for a list of exchanges, for the last year</h1>

In [43]:
with open('targets_lists/exchange_list.csv', newline=None) as f:
    reader = csv.reader(f)
    exchange_list = list(reader)

In [44]:
params_dict = {'days' : 365}

In [45]:
request_url = f"/exchanges/{exchange_list[0][0]}/volume_chart"
response = requests.get(base_url + request_url, params = params_dict)
if response.status_code != 200:
    print(f"API error on {exchange_list[0][0]}")
else:
    exchange_data = response.json()
    exchange_volume_data_df = pd.DataFrame(data = exchange_data, index=None, columns=["Date", exchange_list[0][0]])
    print(f"Successfully saved {exchange_list[0][0]} data")

if len(exchange_list[0]) > 0:
    for exchange_id in exchange_list[0][1:]:
        request_url = f"/exchanges/{exchange_id}/volume_chart"
        response = requests.get(base_url + request_url, params = params_dict)
        if response.status_code != 200:
            print(f"API error on {exchange_id}")
        else:
            exchange_data = response.json()
            exchange_v = [day[1] for day in exchange_data]
            exchange_volume_data_df[[f'{exchange_id}']] = pd.DataFrame(exchange_v, index=None)
            print(f"Successfully saved {exchange_id} data")

exchange_volume_data_df.to_csv(f"exchange_data/exchange_volume_data.csv", index = False)

Successfully saved binance data
Successfully saved huobi data
Successfully saved kraken data


<h1>Plot some data from the previously saved CSVs</h1>